<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/09_Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP(Naturala Language Processing) and NLU (Natural Language Understanding) :

`A handful of example natural language processing (NLP) and natural language understanding (NLU) problems. These are also often referred to as sequence problems (going from one sequence to another)`

NLP has the goal of deriving information out of natural language (Could be sequences text or speech). Another common term of NLP probelems is sequence to sequence problems (seq2seq)

**Let's see where do we use it : Which are reffered to as a Sequence Problems**

1. MultiLabel Classification
2. Machine Translation
3. Text Generation
4. Voice Assistant

**Types of Sequence problems :(input to output types)**
1. One to one  (one word translation)
2. One to many (Image captioning, possible to have multiple captions)
3. Many to one (Sentiment analysis maybe a youtube comment, BitCoin Price Prediction)
4. many to many (Part-of-speech tagging in natural language processing. Each word in the input sentence may correspond to multiple possible part-of-speech tags.)
5. many to many (DNA sequence alignment. Aligning two DNA sequences where each base in one sequence may correspond to multiple bases in the other.)







# What will we do in this section :         

* Downloading and preparing the text data
* How to prepare text data for modeling (Tokenization and embedding)
* Setting up multiple modeling experiments with Recurrent Neural Networks (RNN)
* Building a text feature extraction model using TensorFlow Hub
* Finding the most wrong prediction examples
* Using a model we've built to make predictions on the text from the wild



**NLP Inputs and outputs :**

`Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)`

esource: For a great overview of NLP and the different problems within it, read the article A Simple Introduction to [Natural Language Processing](https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32).



# STEPS  WE FOLLOW IN CREATING IN RNN / CNN FOR NLP

1. Get the data ready (Turn all data into numbers as neural networks can't handle text or natural language and make sure all of your tensors are in right shape pad sequence which don't fit )
2. Build or pick a pretrained model (to suit your problem)
3. Fit the model to the data and make a prediction
4. Evaluate the model
5. Improve through Experimentation
6. Save and reload your trained model



# TYPICAL ARCHITECTURE OF A RECCURENT NEURAL NETWORK

**What is RNN :**  is class of Artificial Neural Networks where connections between nodes from a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behaviour.

1. Input Text
2. Input Layers
3. Text vectorization layer
4. Embedding (Turns mapping of text vectors to embedding matrix (representing how words relate Ex King-man+woman = queen))
5. RNN Cells (Finds Pattern in sequence) `Ex : Simple RNN, LSTM, GRU`
6. Hidden Activations : Adds non linearity to learned features usually use `tanh`
7. Pooling layer : Reduces the dimensionality of the learned features usually Conv1D models `Ex : GlobalAveragePooling1D or GlobalMaxPool1D`
8. Fully Connected layer : Further refines learned features from recurrent layers (Dense layer)
9. Output layer (Takes learned feature and outputs them in shape of target labels) Ex : `output_shape = [number_of_classes] (Disaster or not)`
10. Output Activation : Adds non-linearities to output layer.






# Check GPU

In [ ]:
!nvidia-smi

Thu Dec 28 20:14:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get helper functions

In [ ]:
# Get helper functions Import the series of the helper functions using the below link

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Import the functions using import
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-28 20:14:45--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2023-12-28 20:14:45 (12.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Get the Text Dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as diaster or not diaster)

See the original source here :https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip



In [ ]:
#Get the data

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip the data
unzip_data("nlp_getting_started.zip")

--2023-12-28 20:14:49--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.207, 108.177.97.207, 142.250.157.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   965KB/s    in 0.6s    

2023-12-28 20:14:50 (965 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Becoming one with the data and visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use python: https://realpython.com/python-csv/#:~:text=Reading%20from%20a%20CSV%20file,which%20does%20the%20heavy%20lifting.


 but I prefer to get visual straight way

 So another way to do this to use pandas.



In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [ ]:
#Shuffle training dataframe
train_df_shuffled = train_df.sample(frac =1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
#What does the test dataframe look like

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
#How many total samples
len(train_df) + len(test_df)

10876

In [ ]:
#Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create a random indexes not higher than the total num
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target : {target}", "(real diaster)" if target >0 else "(not diaster)")
  print(f"Text :  {text}\n")
  print("---\n")

Target : 0 (not diaster)
Text :  @emmerdale is Ross really dead?? #AskCharley

---

Target : 1 (real diaster)
Text :  Photo: blue by @forest.fires source: http://t.co/awXR24zsqh http://t.co/o9A26Fn27y

---

Target : 0 (not diaster)
Text :  I added a video to a @YouTube playlist http://t.co/f2TqMFh1Yb Cher Lloyd - Sirens

---

Target : 0 (not diaster)
Text :  '60 all out? What!' - World reacts to Aussie collapse http://t.co/I6zQlk2Puz

---

Target : 1 (real diaster)
Text :  Army veteran fears loose rocks from Dundee landslide could kill him at any moment: He has faced a hail of bulletsÛ_ http://t.co/sxmLg3XdvX

---



# Split data into training and validation sets



In [ ]:
from sklearn.model_selection import train_test_split



In [ ]:
# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, # use 10% of training data for validation
                                                                            random_state = 42)

In [ ]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
#Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with the text problems, one of the first things you'll have to do before you can build a model is to convert text to numbers.

`There are few ways to do this, namely :`  
**Tokenization :** Straight mapping from token to number (Can be modelled but quickly get's too big)

**Ebedding :** Richer representation of relationships between tokens (Can limit size + can be learned) : Create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeding can be learned)





# Text Vectorization (Tokenization)

In [ ]:
train_sentences[:5]


array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
# Text Vectorization (Tokenization)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=10000, # How many words in the vocabulary (automatically add  <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams = None, # Create groups of n-words
                                    output_mode = "int", # how to map tokens to numbers
                                    output_sequence_length = None, #how long do you want your sequences to be
                                    pad_to_max_tokens = True)

In [ ]:
len(train_sentences[0].split())

7

In [ ]:
#Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
#Setup text vectorization variables
max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [ ]:
#Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)


In [ ]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [ ]:
#Create a sample sentence and tokenize it
sample_sentance = "There is a flood in my street"
text_vectorizer([sample_sentance])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from a training dataset and tokenize it

random_sentence = random.choice(train_sentences)
print(f"Original text : {random_sentence}\
        \n\nVectorized version :")
text_vectorizer([random_sentence])


Original text : 70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/aG65u29SGo        

Vectorized version :


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 325,  141,   43,  210, 1152,  224,   80, 1467,   14,  122,  501,
           2,  521,    6,    2]])>

In [ ]:
#Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get the all of the unique words in the
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocabulary : {len(words_in_vocab)}")
print(f"Top 5 words in vocabulary : {top_5_words}")
print(f"Bottom 5 words in vocabulary : {bottom_5_words}")

Number of words in vocabulary : 10000
Top 5 words in vocabulary : ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 words in vocabulary : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating an Embedding using an Embedding Layer

Turns a positive integers (indexes) into dense vectors of fixed size

To make our embeddding,  we're going to use tensorflow layer and for more infor please visit the below link : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about  for our embedding layer :      

* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented 100 long.
* `input_length`  = Length of the sequences being passes to the embedding layer



In [ ]:
from tensorflow.keras import layers

#Create a embedding layer
embedding = layers.Embedding(input_dim = max_vocab_length, # Set input shape
                             output_dim = 128, # Output shape
                             input_length = max_length) # How long is each input
embedding

In [ ]:
#Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text : {random_sentence}\
      \n \n Embedded Version :" )

#Embed the random sentences (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text : Visting Georgina at The Drowned Rat Cafe.....awful view!! http://t.co/pYLFb3tI9U      
 
 Embedded Version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00164849, -0.03810613,  0.00080868, ..., -0.00961839,
         -0.01688312,  0.03940514],
        [-0.00129775,  0.02682105, -0.00815652, ..., -0.00341357,
          0.01427759,  0.01191406],
        [-0.04147561,  0.0214853 ,  0.0467618 , ..., -0.02178475,
          0.03627222, -0.00899567],
        ...,
        [ 0.03057302,  0.01039114, -0.04023448, ..., -0.02686076,
         -0.03972206, -0.03152286],
        [ 0.03057302,  0.01039114, -0.04023448, ..., -0.02686076,
         -0.03972206, -0.03152286],
        [ 0.03057302,  0.01039114, -0.04023448, ..., -0.02686076,
         -0.03972206, -0.03152286]]], dtype=float32)>

In [ ]:
#Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00164849, -0.03810613,  0.00080868, -0.03819388,  0.04523058,
        -0.03666168,  0.034865  , -0.04397758, -0.00061142, -0.04503129,
        -0.04679053,  0.04618564, -0.00470491, -0.00078817,  0.03010743,
        -0.04668323,  0.01439157,  0.01653304, -0.01044976,  0.03896615,
        -0.03303627,  0.03942814,  0.02307106, -0.0234768 , -0.04472054,
         0.03762576,  0.00436968, -0.00725776, -0.01990583,  0.04626078,
         0.04513207,  0.03298623, -0.0282859 ,  0.04790293, -0.02782922,
        -0.01679943,  0.03223384,  0.03068993,  0.01995071,  0.04818558,
         0.04065147,  0.0151688 ,  0.0247568 , -0.00183063,  0.04863146,
        -0.00774169, -0.02296449, -0.03238713,  0.04553488,  0.01853899,
         0.03919646, -0.02601526, -0.01996811, -0.0026335 , -0.0076161 ,
        -0.01587592,  0.04299532, -0.0095307 ,  0.04539898,  0.02025225,
        -0.0489847 ,  0.01311684, -0.0380102 ,  0.04438731,  0.04585465,
  

## Modeling a text dataset (Running a series of experiments)

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

`Model 0`: Naive Bayes  TF-IDF (baseline) [Sklearn ML Map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

`Model 1`: Feed-forward neural network (dense model)

`Model 2`: LSTM model

`Model 3`: GRU model

`Model 4`: Bidirectional-LSTM model

`Model 5`: 1D Convolutional Neural Network

`Model 6`: TensorFlow Hub Pretrained Feature Extractor (Using transfer learning for NLP)

`Model 7`: Same as model 6 with 10% of training data


Model 0 is the simplest to acquire a baseline which we'll expect each other of the other deeper models to beat.

**Each experiment will go through the following steps:**

*   Construct the model
*   Train the model
*   Make predictions with the model
*   Track prediction evaluation metrics for later comparison

      
      
      
      

# Model 0 : Getting a baseline

As with all machine learning modelling experiments, it's importat to create baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

**Note :** It's common practice to use non-DL algorithms as baseline because of their speed and then later using DL to see if you can improve upon them.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modeling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # Convert words to numbers using tfidf
    ('clf', MultinomialNB()) # model the text
])

#Fit the pipeline to training the data
model_0.fit(train_sentences, train_labels)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our Baseline model achieves an accuracy : {round(baseline_score * 100)}%")

Our Baseline model achieves an accuracy : 79%


In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# Creating an evaluation function for our experiments
